In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing import image
tf.__version__

'2.14.0'

##TASK 1##

In [2]:
# Part 1 - Data Preprocessing
data = pd.read_csv("churn_modelling.csv")
print(data)
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

data = pd.get_dummies(data, columns=["Geography"], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(data.drop('Exited', axis=1), data['Exited'], test_size=0.2, random_state=42)

      RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMemb

In [21]:
# Part 2 - Building the ANN
input_size = X_train.shape[1]
output_size = 1
hidden_layer1_size = 12
hidden_layer2_size = 0

def InitializePara(input_size, HL1S, HL2S, outpu_size):
  np.random.seed(0)
  W1 = np.random.rand(input_size, HL1S)
  B1 = np.zeros((1, HL1S))

  W2 = np.random.rand(HL1S, HL2S) if HL2S else None
  B2 = np.zeros((1, HL2S)) if HL2S else None

  W3 = np.random.rand(HL2S, output_size) if HL2S else np.random.rand(HL1S, output_size)
  B3 = np.zeros((1, output_size))

  return W1, B1, W2, B2, W3, B3

def forwardProp(X, W1, B1):
  X = np.array(X, dtype=float)
  L1_out = np.dot(X, W1) + B1
  L1_act = sigmoid(L1_out)

  return L1_out, L1_act


def addHiddenLayer(X, W, B):
  L_out = np.dot(X, W) + B
  L_act = sigmoid(L_out)

  return L_out, L_act

def outputLayer(X, W, B):
  output = np.dot(X, W) + B
  output_act = sigmoid(output)

  return output, output_act

def sigmoid(x):
  return 1/ (1 + np.exp(-x))

W1, B1, W2, B2, W3, B3 = InitializePara(input_size, hidden_layer1_size, hidden_layer2_size, output_size)
L1_out, L1_act = forwardProp(X_train, W1, B1)

if(hidden_layer2_size):
  L2_out, L2_act = addHiddenLayer(L1_act, W2, B2)

output, output_act = outputLayer(L2_act if hidden_layer2_size else L1_act, W3, B3)

In [25]:
# print("W1:\n", W1,"\nB1:\n", B1)
# print("W2:\n", W2,"\nB2:\n", B2)
# print("W3:\n", W3,"\nB3:\n", B3)

print(L1_out, L1_act)
print(output, output_act)

[[105426.88629323   4117.94156125 148883.96385356 ... 103473.27183488
  106578.47442516 102872.412653  ]
 [153034.04511846  47933.7228983  231058.11813323 ... 143486.15925271
  172315.59305465 141752.59381468]
 [ 86882.3086164   43878.34431682 136968.47561353 ...  78780.99708069
  104801.32263331  77509.9054697 ]
 ...
 [ 54497.25458778   2405.24956483  76896.34021041 ...  53437.68745838
   55183.94962328  53186.88221707]
 [117570.60566472  71659.54774957 189712.62484038 ... 104649.86337696
  146966.03591586 102677.92554612]
 [ 78472.79424757  55408.55334093 129184.34497016 ...  68635.13034454
  101250.12215346  67218.05759125]] [[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[[6.49753858]
 [6.49753858]
 [6.49753858]
 ...
 [6.49753858]
 [6.49753858]
 [6.49753858]] [[0.99849512]
 [0.99849512]
 [0.99849512]
 ...
 [0.99849512]
 [0.99849512]
 [0.99849512]]


In [29]:
# Part 3 - Training the ANN
X_train = X_train.astype('float32')
y_train = y_train.astype('int32')
classifier = Sequential()

classifier.add(Dense(units = hidden_layer1_size, kernel_initializer = 'uniform', activation = 'relu', input_dim = input_size))

if(hidden_layer2_size):
    classifier.add(Dense(units = hidden_layer2_size, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = output_size, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 2.2056 - accuracy: 0.6981
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.8954 - accuracy: 0.7254
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.6553 - accuracy: 0.7510
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7856
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5253 - accuracy: 0.7921
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5181 - accuracy: 0.7945
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5193 - accuracy: 0.7945
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5220 - accuracy: 0.7945
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5073 - accuracy: 0.7945
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.5039 - accura

In [34]:
# Part 4 - Making the predictions and evaluating the model

X_test = X_test.astype('float32')
y_test = y_test.astype('int32')

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

conf_mat = confusion_matrix(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_mat)
print("Accuracy: ", accuracy)


63/63 [==============================] - 0s 2ms/step
Confusion Matrix:
[[1607    0]
 [ 393    0]]
Accuracy:  0.8035


##TASK 2##

In [3]:
def preprocess(path, Type, class_=[], classification='binary'):
    
    if(Type == "test"):
        datagen = ImageDataGenerator(
            rescale = 1./255
        )
    else:
        datagen = ImageDataGenerator(
            rescale = 1./255,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True
        )
    
    set_ = datagen.flow_from_directory(
        path,
        target_size = (64, 64),
        class_mode = classification,
        classes = class_
    )
    
    return set_

In [7]:

import os
import tensorflow as tf

# Set the number of threads to the number of cores available
os.environ['TF_NUM_INTEROP_THREADS'] = str(os.cpu_count())
os.environ['TF_NUM_INTRAOP_THREADS'] = str(os.cpu_count())

# Configure tensorflow to use the set number of threads
config = tf.compat.v1.ConfigProto()
config.inter_op_parallelism_threads = os.cpu_count()
config.intra_op_parallelism_threads = os.cpu_count()
session = tf.compat.v1.Session(config=config)

# Part 1 - Data Preprocessing
train_dataset = 'dataset/training_set'
test_dataset = 'dataset/test_set'
prediction = 'dataset/single_prediction'
pred_size = 2

train_set = preprocess(
    train_dataset,
    "train",
    ['cats', 'dogs'],
    'binary'
)

test_set = preprocess(
    test_dataset,
    "test",
    ['cats', 'dogs'],
    'binary'
)


# Part 2 - Building the CNN
cnn = Sequential()

cnn.add(
    Conv2D(32, (3, 3),
    input_shape=(64, 64, 3),
    activation = 'relu')
)

cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(
    Conv2D(32, (3, 3),
    activation='relu')
)
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())

cnn.add(Dense(units=128, activation='relu'))

cnn.add(Dense(units=1, activation='sigmoid'))

# part 3 - Training the CNN
cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

cnn.fit(
    x = train_set,
    validation_data=test_set,
    epochs=25
)

# Part 4 - Making a single prediction
for i in range(1, pred_size + 1):
    test_image = image.load_img(
        prediction + f'/cat_or_dog_{i}.jpg',
        target_size = (64, 64)
    )

    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn.predict(test_image)
    
    if result[0][0] == 1:
        prediction_output = 'dog'
    else:
        prediction_output = 'cat'
    
    print(prediction_output)


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 30s 117ms/step - loss: 0.6636 - accuracy: 0.5885 - val_loss: 0.5879 - val_accuracy: 0.6820
Epoch 2/25
250/250 [==============================] - 24s 96ms/step - loss: 0.5920 - accuracy: 0.6875 - val_loss: 0.6482 - val_accuracy: 0.6575
Epoch 3/25
250/250 [==============================] - 32s 126ms/step - loss: 0.5484 - accuracy: 0.7245 - val_loss: 0.5207 - val_accuracy: 0.7365
Epoch 4/25
250/250 [==============================] - 34s 135ms/step - loss: 0.5258 - accuracy: 0.7376 - val_loss: 0.4929 - val_accuracy: 0.7630
Epoch 5/25
250/250 [==============================] - 37s 148ms/step - loss: 0.4858 - accuracy: 0.7639 - val_loss: 0.4958 - val_accuracy: 0.7595
Epoch 6/25
250/250 [==============================] - 37s 146ms/step - loss: 0.4739 - accuracy: 0.7747 - val_loss: 0.4645 - val_accuracy: 0.7900
Epoch 7/25
250/250 [===========================